# Learning plurality voting with DeepSets

## Dataset construction

In [32]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch_geometric.nn import MLP, DeepSetsAggregation, conv
from tqdm import tqdm

from geometric_governance.util import RangeOrValue, get_value
from geometric_governance.data import generate_synthetic_election

In [403]:
NUM_VOTERS_RANGE = (3, 10)
NUM_COLLUDERS_RANGE = (0, 4)
NUM_CANDIDATES = 5
TRAIN_BATCH_SIZE = 128
TRAIN_NUM_EPOCHS = 1_000
NUM_COLLUDE_STEPS = 1
NUM_DISCRIMINATOR_STEPS = 1

In [404]:
def fully_connected_directed_edge_index(n):
    row, col = torch.meshgrid(torch.arange(n), torch.arange(n), indexing='ij')
    edge_index = torch.stack([row.flatten(), col.flatten()], dim=0)
    return edge_index[:, edge_index[0] != edge_index[1]]  # Remove self-loops

print(fully_connected_directed_edge_index(5))

tensor([[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
        [1, 2, 3, 4, 0, 2, 3, 4, 0, 1, 3, 4, 0, 1, 2, 4, 0, 1, 2, 3]])


In [405]:
class CollusionModel(nn.Module):
    def __init__(self, num_candidates: int, embedding_size: int = 128):
        super().__init__()
        self.embedding_size = embedding_size

        self.fc1 = nn.Linear(num_candidates, embedding_size)
        self.convs = nn.ModuleList([conv.GATv2Conv(in_channels=(2 * embedding_size), out_channels=(embedding_size)) for _ in range(5)])
        self.fc2 = nn.Linear(embedding_size, num_candidates)

    def forward(self, x):
        index = fully_connected_directed_edge_index(x.size(-2))
        
        x = self.fc1(x)
        
        for conv_layer in self.convs:
            # Use Randomized Normal Features to disambiguate voters
            rnf = torch.normal(mean=torch.zeros_like(x), std=1.)
            x = torch.concat([x, rnf], dim=-1)

            x = conv_layer(x, index)
            
        x = F.relu(x)
        
        x = self.fc2(x)
        
        x = F.softmax(x, dim=-1)
        
        return x

In [406]:
class DiscriminatorModel(nn.Module):
    def __init__(self, num_candidates: int, embedding_size: int = 128):
        super().__init__()
        self.embedding_size = embedding_size

        self.fc1 = nn.Linear(num_candidates, embedding_size)
        self.convs = nn.ModuleList([conv.GATv2Conv(in_channels=(2 * embedding_size), out_channels=(embedding_size)) for _ in range(3)])
        self.fc2 = nn.Linear(embedding_size, 1)

    def forward(self, x):
        index = fully_connected_directed_edge_index(x.size(-2))
        
        x = self.fc1(x)
        
        for conv_layer in self.convs:
            # Use Randomized Normal Features to disambiguate voters
            rnf = torch.normal(mean=torch.zeros(x.size(-2), self.embedding_size), std=1.)
            x = torch.concat([x, rnf], dim=-1)
            
            x = conv_layer(x, index)

        x = F.relu(x)
        
        x = self.fc2(x)
        
        x = torch.sigmoid(x)

        x = x.view(-1)
        
        return x

In [407]:
collusion_model = CollusionModel(NUM_CANDIDATES)
collusion_model.train()
collusion_optim = torch.optim.Adam(collusion_model.parameters())

In [408]:
discriminator_model = DiscriminatorModel(NUM_CANDIDATES)
discriminator_model.train()
discriminator_optim = torch.optim.Adam(discriminator_model.parameters())

In [409]:
rng = np.random.default_rng(seed=42)

epochs = tqdm(range(TRAIN_NUM_EPOCHS))
for epoch in epochs:
    for _ in range(NUM_COLLUDE_STEPS):
        num_voters = get_value(NUM_VOTERS_RANGE, rng)
        num_colluders = get_value(NUM_COLLUDERS_RANGE, rng)
        voter_utilities = rng.dirichlet(
            alpha=(1.,) * NUM_CANDIDATES, size=(num_voters + 1)
        )
        # Duplicate the last candidate's data to generate colluding voters' data
        # so that all colluding candidates share the same utility
        collusion_utility = voter_utilities[-1][None, :]
        collusion_candidate = np.argmax(collusion_utility)
        collusion_utilities = np.tile(collusion_utility, (num_colluders, 1))
        collusion_utilities = torch.from_numpy(collusion_utilities).float()
    
        # Colluders cast their vote jointly (by adjusting the utilities)
        collusion_utilities = collusion_model(collusion_utilities)
    
        # Assimilate votes, removing those thought to be fake by the discriminator
        assimiliated_utilities = torch.cat([torch.from_numpy(voter_utilities[:-1]).float(), collusion_utilities])
        possibly_fake = 1. - discriminator_model(assimiliated_utilities).detach()
        assimiliated_utilities = assimiliated_utilities * possibly_fake.unsqueeze(-1)
    
        # Plurality voting
        election_result = F.softmax(torch.sum(assimiliated_utilities, dim=-2), dim=-1)
    
        # The colluders' loss is how close they can get to getting their chosen candidate to win the election/lottery
        ideal_result = torch.zeros_like(election_result)
        ideal_result[collusion_candidate] = 1.
    
        collusion_optim.zero_grad()
        collusion_loss = F.cross_entropy(election_result, ideal_result)
        collusion_loss.backward()
        collusion_optim.step()

    for _ in range(NUM_DISCRIMINATOR_STEPS):
        num_voters_discrim = get_value(NUM_VOTERS_RANGE, rng)
        num_colluders_discrim = get_value(NUM_COLLUDERS_RANGE, rng)
        voter_utilities_discrim = rng.dirichlet(
            alpha=(1.,) * NUM_CANDIDATES, size=(num_voters_discrim + 1)
        )
        # Duplicate the last candidate's data to generate colluding voters' data
        # so that all colluding candidates share the same utility
        collusion_utility_discrim = voter_utilities_discrim[-1][None, :]
        collusion_utilities_discrim = np.tile(collusion_utility_discrim, (num_colluders_discrim, 1))
        collusion_utilities_discrim = torch.from_numpy(collusion_utilities_discrim).float()
        
        # Discriminator tries to distinguish the colluders' voting patterns
        # Regenerate colluders' output
        collusion_utilities_discrim = collusion_model(collusion_utilities_discrim).detach()
        assimiliated_utilities_discrim = torch.cat([torch.from_numpy(voter_utilities_discrim[:-1]).float(), collusion_utilities_discrim])
    
        discriminator_output = discriminator_model(assimiliated_utilities_discrim)
        colluders_mask = torch.cat([torch.zeros(num_voters_discrim), torch.ones(num_colluders_discrim)])
        discriminator_optim.zero_grad()
        discriminator_loss = F.binary_cross_entropy(discriminator_output, colluders_mask)
        discriminator_loss.backward()
        discriminator_optim.step()

    epochs.set_postfix({"collusion_loss": collusion_loss, "discriminator_loss": discriminator_loss})
    


100%|█████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:07<00:00, 132.49it/s, collusion_loss=tensor(1.1485, grad_fn=<DivBackward1>), discriminator_loss=tensor(0.0085, grad_fn=<BinaryCrossEntropyBackward0>)]


In [410]:
collusion_utility

array([[0.02813386, 0.06883993, 0.00408478, 0.56029761, 0.33864382]])

In [411]:
collusion_utilities

tensor([[0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.]], grad_fn=<SoftmaxBackward0>)

In [412]:
possibly_fake

tensor([0.7189, 0.7173, 0.7182, 0.7210, 0.7177, 0.7181, 0.7139, 0.7219])

In [413]:
election_result

tensor([0.0540, 0.0500, 0.1309, 0.6958, 0.0693], grad_fn=<SoftmaxBackward0>)

In [414]:
ideal_result

tensor([0., 0., 0., 1., 0.])

In [415]:
discriminator_output

tensor([0.0085, 0.0084, 0.0085, 0.0084, 0.0086], grad_fn=<ViewBackward0>)

In [416]:
colluders_mask

tensor([0., 0., 0., 0., 0.])

In [417]:
assimiliated_utilities_discrim

tensor([[0.1059, 0.0349, 0.1458, 0.5873, 0.1261],
        [0.0092, 0.0398, 0.5392, 0.0300, 0.3818],
        [0.6413, 0.0779, 0.0100, 0.0861, 0.1846],
        [0.1527, 0.3545, 0.1766, 0.1637, 0.1524],
        [0.0429, 0.0893, 0.2626, 0.3948, 0.2103]])